In [1]:
import numpy as np
import pandas as pd
import ast
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv("C:/Users/abhil/OneDrive/Desktop/Real_Estate_Project/Real_Estate_ML_Project/data/raw/appartments.csv").drop(22)

In [3]:
df.head()

,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Salon', 'Restaurant', 'Spa'..."
1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...","['Bowling Alley', 'Mini Theatre', 'Manicured G..."
2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,"['Terrace Garden', 'Gazebo', 'Fountain', 'Amph..."
3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'The Shikshiyan School': '2.9 KM', 'WTC Plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...","['Swimming Pool', 'Volley Ball Court', 'Aerobi..."
4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'Pranavananda Int. School': '450 m', 'DLF Sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,"['Mini Theatre', 'Doctor on Call', 'Concierge ..."


In [4]:
df['facilitiesStr'] = (df['TopFacilities'].apply(ast.literal_eval)).apply(' '.join)

In [5]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english',ngram_range=(1,2))

In [6]:
tfidf_matrix = tfidf_vectorizer.fit_transform(df['facilitiesStr'])

In [7]:
tfidf_matrix[0].toarray()

array([[0.        , 0.        , 0.        , 0.18809342, 0.18809342,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [8]:
cosine_sim1 = cosine_similarity(tfidf_matrix,tfidf_matrix)

In [9]:
cosine_sim1.shape

(246, 246)

In [10]:
def recommended_properties(property_name,cosie_sim=cosine_sim1):
    indx = df[df['PropertyName'] == property_name].index.tolist()[0]
    sim_scores = list(enumerate(cosine_sim1[indx]))
    sim_scores = sorted(sim_scores,key=lambda x:x[1],reverse=True)
    sim_scores = sim_scores[1:6]
    print(f'printing similiarity score->{sim_scores}')
    property_index = [i[0] for i in sim_scores]
    recommended_df = pd.DataFrame({
        "Property_Name" : df['PropertyName'].iloc[property_index],
        "similarity_score" : sim_scores
    })
    return recommended_df

In [11]:
recommended_properties("DLF The Arbour")

printing similiarity score->[(63, np.float64(0.551340635885514)), (92, np.float64(0.48399273152019534)), (216, np.float64(0.4791925565137415)), (18, np.float64(0.46907955257042044)), (62, np.float64(0.4160816171209393))]


,Property_Name,similarity_score
64,Ace Palm Floors,"(63, 0.551340635885514)"
93,JMS The Nation,"(92, 0.48399273152019534)"
217,Yashika 104,"(216, 0.4791925565137415)"
18,Whiteland Blissville,"(18, 0.46907955257042044)"
63,Vatika Aspiration,"(62, 0.4160816171209393)"


In [12]:
df_appartments = pd.read_csv("C:/Users/abhil/OneDrive/Desktop/Real_Estate_Project/Real_Estate_ML_Project/data/raw/appartments.csv").drop(22)

In [13]:
def refined_parse_modified_v2(detail_str):
    try:
        details = json.loads(detail_str.replace("'", "\""))
    except:
        return {}

    extracted = {}
    for bhk, detail in details.items():
        # Extract building type
        extracted[f'building type_{bhk}'] = detail.get('building_type')

        # Parsing area details
        area = detail.get('area', '')
        area_parts = area.split('-')
        if len(area_parts) == 1:
            try:
                value = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area low {bhk}'] = value
                extracted[f'area high {bhk}'] = value
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None
        elif len(area_parts) == 2:
            try:
                extracted[f'area low {bhk}'] = float(area_parts[0].replace(',', '').replace(' sq.ft.', '').strip())
                extracted[f'area high {bhk}'] = float(area_parts[1].replace(',', '').replace(' sq.ft.', '').strip())
            except:
                extracted[f'area low {bhk}'] = None
                extracted[f'area high {bhk}'] = None

        # Parsing price details
        price_range = detail.get('price-range', '')
        price_parts = price_range.split('-')
        if len(price_parts) == 2:
            try:
                extracted[f'price low {bhk}'] = float(price_parts[0].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                extracted[f'price high {bhk}'] = float(price_parts[1].replace('₹', '').replace(' Cr', '').replace(' L', '').strip())
                if 'L' in price_parts[0]:
                    extracted[f'price low {bhk}'] /= 100
                if 'L' in price_parts[1]:
                    extracted[f'price high {bhk}'] /= 100
            except:
                extracted[f'price low {bhk}'] = None
                extracted[f'price high {bhk}'] = None

    return extracted

In [14]:
data_refined = []

for _, row in df_appartments.iterrows():
    features = refined_parse_modified_v2(row['PriceDetails'])
    
    # Construct a new row for the transformed dataframe
    new_row = {'PropertyName': row['PropertyName']}
    
    # Populate the new row with extracted features
    for config in ['1 BHK', '2 BHK', '3 BHK', '4 BHK', '5 BHK', '6 BHK', '1 RK', 'Land']:
        new_row[f'building type_{config}'] = features.get(f'building type_{config}')
        new_row[f'area low {config}'] = features.get(f'area low {config}')
        new_row[f'area high {config}'] = features.get(f'area high {config}')
        new_row[f'price low {config}'] = features.get(f'price low {config}')
        new_row[f'price high {config}'] = features.get(f'price high {config}')
    
    data_refined.append(new_row)

df_final_refined_v2 = pd.DataFrame(data_refined).set_index('PropertyName')

In [15]:
df_final_refined_v2['building type_Land']= df_final_refined_v2['building type_Land'].replace({'':'Land'})

In [16]:
df_final_refined_v2.shape

(246, 40)

In [17]:
categorical_columns = df_final_refined_v2.select_dtypes(include=['object']).columns.tolist()

In [18]:
categorical_columns

['building type_1 BHK',
 'building type_2 BHK',
 'building type_3 BHK',
 'building type_4 BHK',
 'building type_5 BHK',
 'building type_6 BHK',
 'building type_1 RK',
 'building type_Land']

In [19]:
ohe_df = pd.get_dummies(df_final_refined_v2,columns=categorical_columns,drop_first=True)

In [20]:
ohe_df.shape

(246, 43)

In [21]:
ohe_df.fillna(0,inplace=True)

In [22]:
ohe_df

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,...,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,0.0,0.0,0.00,0.0000,1370.0,1370.0,2.0000,2.40,1850.0,2050.0,...,False,False,False,False,False,False,False,False,False,False
M3M Crown,0.0,0.0,0.00,0.0000,0.0,0.0,0.0000,0.00,1605.0,2170.0,...,False,False,False,False,False,False,False,False,False,False
Adani Brahma Samsara Vilasa,0.0,0.0,0.00,0.0000,0.0,0.0,0.0000,0.00,1800.0,3150.0,...,False,False,True,False,False,True,False,False,False,False
Sobha City,0.0,0.0,0.00,0.0000,1381.0,1692.0,1.5500,3.21,1711.0,2343.0,...,False,False,False,False,False,False,False,False,False,False
Signature Global City 93,0.0,0.0,0.00,0.0000,981.0,1118.0,0.9301,1.06,1235.0,1530.0,...,True,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,0.0,0.0,0.00,0.0000,964.0,964.0,0.0000,0.00,1127.0,1127.0,...,False,False,False,False,False,False,False,False,False,False
Pyramid Urban Homes 2,335.0,398.0,23.45,0.2786,500.0,625.0,0.0000,0.00,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
Satya The Hermitage,0.0,0.0,0.00,0.0000,1450.0,1450.0,0.0000,0.00,1991.0,1991.0,...,False,False,False,False,False,False,False,False,False,False


In [23]:
scaler = StandardScaler()

ohe_df_normalized = pd.DataFrame(scaler.fit_transform(ohe_df),columns=ohe_df.columns,index=ohe_df.index)

In [24]:
ohe_df_normalized

,area low 1 BHK,area high 1 BHK,price low 1 BHK,price high 1 BHK,area low 2 BHK,area high 2 BHK,price low 2 BHK,price high 2 BHK,area low 3 BHK,area high 3 BHK,...,building type_2 BHK_Independent Floor,building type_2 BHK_Service Apartment,building type_3 BHK_Independent Floor,building type_3 BHK_Service Apartment,building type_3 BHK_Villa,building type_4 BHK_Independent Floor,building type_4 BHK_Villa,building type_5 BHK_Independent Floor,building type_5 BHK_Villa,building type_6 BHK_Villa
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-0.252266,-0.169584,-0.105197,-0.082332,1.223499,1.020101,-0.173712,1.158423,0.553787,0.370864,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
M3M Crown,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.293086,0.472749,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Adani Brahma Samsara Vilasa,-0.252266,-0.169584,-0.105197,-0.082332,-0.893541,-0.896660,-0.283546,-0.387986,0.500583,1.304803,...,-0.289310,-0.063888,2.683282,-0.063888,-0.171139,3.924283,-0.236208,-0.111111,-0.216353,-0.063888
Sobha City,-0.252266,-0.169584,-0.105197,-0.082332,1.240497,1.470610,-0.198425,1.680336,0.405879,0.619632,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Signature Global City 93,-0.252266,-0.169584,-0.105197,-0.082332,0.622383,0.667529,-0.232468,0.295011,-0.100626,-0.070634,...,3.456497,-0.063888,2.683282,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,-0.252266,-0.169584,-0.105197,-0.082332,0.596113,0.452068,-0.283546,-0.387986,-0.215547,-0.412795,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Pyramid Urban Homes 2,1.565039,0.704171,9.593528,0.224987,-0.120899,-0.022226,-0.283546,-0.387986,-1.414772,-1.369658,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
Satya The Hermitage,-0.252266,-0.169584,-0.105197,-0.082332,1.347122,1.132029,-0.283546,-0.387986,0.703823,0.320771,...,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888


In [25]:
cosine_sim2 = cosine_similarity(ohe_df_normalized)

In [26]:
cosine_sim2.shape

(246, 246)

In [27]:
type(cosine_sim2)

numpy.ndarray

In [28]:
def recommend_properties_with_scores(property_name):
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim2[ohe_df_normalized.index.get_loc(property_name)]))
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:6]]
    top_scores = [i[1] for i in sorted_scores[1:6]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = ohe_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('M3M Golf Hills')

,PropertyName,SimilarityScore
0,AIPL The Peaceful Homes,0.955462
1,Smartworld One DXP,0.954670
2,Unitech Escape,0.953092
3,M3M Capital,0.951156
4,BPTP Terra,0.943128


In [29]:
df[['PropertyName','LocationAdvantages']]['LocationAdvantages'][0]

"{'Bajghera Road': '800 Meter', 'Palam Vihar Halt': '2.5 KM', 'DPSG Palam Vihar': '3.1 KM', 'Park Hospital': '3.1 KM', 'Gurgaon Railway Station': '4.9 KM', 'The NorthCap University': '5.4 KM', 'Dwarka Expy': '1.2 KM', 'Hyatt Place Gurgaon Udyog Vihar': '7.7 KM', 'Dwarka Sector 21, Metro Station': '7.2 KM', 'Pacific D21 Mall': '7.4 KM', 'Indira Gandhi International Airport': '14.7 KM', 'Hamoni Golf Camp': '6.2 KM', 'Fun N Food Waterpark': '8.8 KM', 'Accenture DDC5': '9 KM'}"

In [30]:
def distance_to_meters(distance_str):
    try:
        if 'Km' in distance_str or "KM" in distance_str:
            return float(distance_str.split()[0]) * 1000
        elif 'Meter' in distance_str or 'meter' in distance_str:
            return float(distance_str.split()[0])
        else:
            return None
    except:
        return None

In [31]:
# Extract distances for each location
location_matrix = {}
for index, row in df.iterrows():
    distances = {}
    for location, distance in ast.literal_eval(row['LocationAdvantages']).items():
        distances[location] = distance_to_meters(distance)
    location_matrix[index] = distances

# Convert the dictionary to a dataframe
location_df = pd.DataFrame.from_dict(location_matrix, orient='index')

# Display the first few rows
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
0,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,550.0,NaN,NaN,NaN,NaN,6700.0,3800.0,NaN,NaN,7500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,5300.0,NaN,NaN,NaN,2500.0,8800.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,1500.0,NaN,NaN,NaN,6500.0,6700.0,5100.0,NaN,NaN,8200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,5500.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
location_df.head()

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
0,800.0,2500.0,3100.0,3100.0,4900.0,5400.0,1200.0,7700.0,7200.0,7400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,550.0,NaN,NaN,NaN,NaN,6700.0,3800.0,NaN,NaN,7500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,5300.0,NaN,NaN,NaN,2500.0,8800.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,1500.0,NaN,NaN,NaN,6500.0,6700.0,5100.0,NaN,NaN,8200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,5500.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
location_df.index = df.PropertyName

In [34]:
location_df.fillna(54000,inplace=True) #filling the missing values as we cannot scale wiht nun value also a large value is taken
#so that it will very unfamilier with other points and cosine similarity will come near to zero not affecting the process

In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

location_df_normalized = pd.DataFrame(scaler.fit_transform(location_df),columns=location_df.columns,index=location_df.index)

In [36]:
location_df_normalized

,Bajghera Road,Palam Vihar Halt,DPSG Palam Vihar,Park Hospital,Gurgaon Railway Station,The NorthCap University,Dwarka Expy,Hyatt Place Gurgaon Udyog Vihar,"Dwarka Sector 21, Metro Station",Pacific D21 Mall,...,MCC Cricket Ground Dhankot,The Shri Ram School Aravali,Taj City Centre Gurugram,Minda Industries Corporate Office,"Rampura Flyover, Naurangpur Rd",Manesar toll plaza - Kherki Daula,"Imt Manesar, Gurugram",Holiday Inn,Sector 84 Road,Skyview Corporate Park
PropertyName,,,,,,,,,,,,,,,,,,,,,
Smartworld One DXP,-7.960979,-15.652476,-15.652476,-3.149592,-2.966108,-3.147217,-3.726615,-10.231739,-15.652476,-6.023233,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
M3M Crown,-7.998993,0.063888,0.063888,0.328277,0.368941,-3.054053,-3.529275,0.090308,0.063888,-6.009941,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Adani Brahma Samsara Vilasa,-7.276720,0.063888,0.063888,0.328277,-3.129124,-2.903557,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Sobha City,-7.854539,0.063888,0.063888,0.328277,-2.857430,-3.054053,-3.430606,0.090308,0.063888,-5.916893,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Signature Global City 93,0.128476,0.063888,0.063888,-2.985606,0.368941,0.335688,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DLF Princeton Estate,0.128476,0.063888,0.063888,0.328277,0.368941,0.335688,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Pyramid Urban Homes 2,0.128476,0.063888,0.063888,0.328277,0.368941,0.335688,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888
Satya The Hermitage,0.128476,0.063888,0.063888,0.328277,0.368941,0.335688,0.280891,0.090308,0.063888,0.171073,...,0.0,0.063888,0.063888,0.063888,0.063888,0.063888,0.063888,0.0,0.063888,0.063888


In [37]:
cosine_sim3 = cosine_similarity(location_df_normalized)

In [38]:
cosine_sim3.shape

(246, 246)

In [41]:
def recommend_properties_with_scores(property_name, top_n=247):
    
    cosine_sim_matrix = 30*cosine_sim1 + 20*cosine_sim2 + 8*cosine_sim3
    # cosine_sim_matrix = cosine_sim3
    
    # Get the similarity scores for the property using its name as the index
    sim_scores = list(enumerate(cosine_sim_matrix[location_df_normalized.index.get_loc(property_name)]))

    print(sim_scores)
    
    # Sort properties based on the similarity scores
    sorted_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices and scores of the top_n most similar properties
    top_indices = [i[0] for i in sorted_scores[1:top_n+1]]
    top_scores = [i[1] for i in sorted_scores[1:top_n+1]]
    
    # Retrieve the names of the top properties using the indices
    top_properties = location_df_normalized.index[top_indices].tolist()
    
    # Create a dataframe with the results
    recommendations_df = pd.DataFrame({
        'PropertyName': top_properties,
        'SimilarityScore': top_scores
    })
    
    return recommendations_df

# Test the recommender function using a property name
recommend_properties_with_scores('Ireo Victory Valley')

[(0, np.float64(8.67940474671015)), (1, np.float64(9.529422107881395)), (2, np.float64(1.9063230360427157)), (3, np.float64(8.928844279537898)), (4, np.float64(1.4733452772174154)), (5, np.float64(15.658830389148173)), (6, np.float64(8.643041134292707)), (7, np.float64(11.316133193634505)), (8, np.float64(-2.738278719954941)), (9, np.float64(4.829835982662244)), (10, np.float64(-3.9066008195826702)), (11, np.float64(-4.329390828418061)), (12, np.float64(-3.9408407683822197)), (13, np.float64(-4.546125203574216)), (14, np.float64(-7.24171847703138)), (15, np.float64(12.292912841862892)), (16, np.float64(8.660616544308601)), (17, np.float64(-7.643827788805829)), (18, np.float64(-9.03328792673062)), (19, np.float64(12.174722679934836)), (20, np.float64(18.438452311014593)), (21, np.float64(7.367770640620103)), (22, np.float64(9.569277514073873)), (23, np.float64(5.7529600023667715)), (24, np.float64(-0.7336418867689661)), (25, np.float64(13.673358489114495)), (26, np.float64(-4.2368365898

,PropertyName,SimilarityScore
0,Pioneer Urban Presidia,28.021460
1,Ambience Creacions,27.787913
2,DLF The Crest,24.205986
3,Pioneer Araya,23.415308
4,SS The Leaf,21.012590
...,...,...
240,JMS The Nation,-14.735656
241,Shree Vardhman City,-14.923561
242,JMS Prime Land,-15.082229
243,Vatika Aspiration,-15.148347


In [42]:
(3*cosine_sim3 + 5*cosine_sim2 + 6*cosine_sim1).shape

(246, 246)

In [47]:
import pickle
with open('C:/Users/abhil/OneDrive/Desktop/Real_Estate_Project/Real_Estate_ML_Project/app/datasets/cosine_sim1.pkl','wb') as fo:
    pickle.dump(cosine_sim1,fo)


In [49]:
with open('C:/Users/abhil/OneDrive/Desktop/Real_Estate_Project/Real_Estate_ML_Project/app/datasets/cosine_sim2.pkl','wb') as fo:
    pickle.dump(cosine_sim2,fo)
with open('C:/Users/abhil/OneDrive/Desktop/Real_Estate_Project/Real_Estate_ML_Project/app/datasets/cosine_sim3.pkl','wb') as fo:
    pickle.dump(cosine_sim3,fo)

In [50]:
with open('C:/Users/abhil/OneDrive/Desktop/Real_Estate_Project/Real_Estate_ML_Project/app/datasets/location_df.pkl','wb') as fo:
    pickle.dump(location_df,fo)